In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
import matplotlib.pyplot as plt
import os
import glob

2024-06-28 13:46:54.623079: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-28 13:46:57.220578: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from os import listdir
from os.path import isfile, join

dataset = pd.concat(map(pd.read_csv, glob.glob('/mnt/c/Users/aardp/Downloads/G3/*.csv')))
dataset.head()

/tmp/ipykernel_33523/1150708094.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset = pd.concat(map(pd.read_csv, glob.glob('/mnt/c/Users/aardp/Downloads/G3/*.csv')))


,Year/Month,Box Size,Longitude,Latitude,PID2: Standard/Enhanced,Group,Checksum,S-S1,S-Median,S-S5,...,R-S1,R-Median,R-S5,R-Mean,R-Observation number,R-Standard deviation,R-Mean day of the month,R-Fraction of daylight observations,R-Mean Longitude,R-Mean Latitude
0,1857/1,2,276.0,6.0,0,3,14,26.10,26.1,26.1,...,65.6,65.6,65.6,65.6,1,0.0,16,1.0,0.8,0.8
0,1858/1,2,352.0,44.0,0,3,7,11.70,11.7,11.7,...,91.6,91.6,91.6,91.6,1,0.0,18,0.0,0.1,0.8
1,1858/1,2,334.0,8.0,0,3,8,25.00,25.6,25.6,...,83.4,83.4,83.4,83.4,1,0.0,26,0.0,0.3,0.2
0,1858/2,2,302.0,24.0,0,3,2,21.10,21.1,21.1,...,58.9,58.9,58.9,58.9,1,0.0,24,0.0,0.6,0.3
1,1858/2,2,332.0,10.0,0,3,4,23.67,23.9,25.7,...,66.4,66.4,66.4,66.4,1,0.0,10,1.0,0.3,0.0


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5078925 entries, 0 to 4466
Data columns (total 47 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   Year/Month                           object 
 1   Box Size                             object 
 2   Longitude                            float64
 3   Latitude                             float64
 4   PID2: Standard/Enhanced              object 
 5   Group                                object 
 6   Checksum                             object 
 7   S-S1                                 float64
 8   S-Median                             float64
 9   S-S5                                 float64
 10  S-Mean                               float64
 11  S-Observation number                 object 
 12  S-Standard deviation                 float64
 13  S-Mean day of the month              object 
 14  S-Fraction of daylight observations  float64
 15  S-Mean Longitude                     flo

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-06-28 13:48:13.348265: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-28 13:48:13.899320: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-28 13:48:13.899390: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.


In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow warnings

# Load the data
df = dataset

# Display the first few rows of the dataframe
print(df.head())

# Display information about the dataframe
print(df.info())

# Display statistical summary
print(df.describe())

# Check for any remaining NaN values
print(df.isna().sum())

# Remove rows with NaN values (if any)
df.dropna(inplace=True)

# Extract temporal features from 'Year/Month'
df['Year/Month'] = pd.to_datetime(df['Year/Month'], format='%Y/%m')
df['Year'] = df['Year/Month'].dt.year
df['Month'] = df['Year/Month'].dt.month

# Drop the original 'Year/Month' column
#df.drop(columns=['Year/Month'], inplace=True)

# Check for any NaN values after pivoting
print(df.isna().sum())

# Handle missing values
df.fillna(method='ffill', inplace=True)

  Year/Month Box Size  Longitude  Latitude PID2: Standard/Enhanced Group  \
0     1857/1        2      276.0       6.0                       0     3   
0     1858/1        2      352.0      44.0                       0     3   
1     1858/1        2      334.0       8.0                       0     3   
0     1858/2        2      302.0      24.0                       0     3   
1     1858/2        2      332.0      10.0                       0     3   

  Checksum   S-S1  S-Median  S-S5  ...  R-S1 R-Median  R-S5 R-Mean  \
0       14  26.10      26.1  26.1  ...  65.6     65.6  65.6   65.6   
0        7  11.70      11.7  11.7  ...  91.6     91.6  91.6   91.6   
1        8  25.00      25.6  25.6  ...  83.4     83.4  83.4   83.4   
0        2  21.10      21.1  21.1  ...  58.9     58.9  58.9   58.9   
1        4  23.67      23.9  25.7  ...  66.4     66.4  66.4   66.4   

   R-Observation number  R-Standard deviation  R-Mean day of the month  \
0                     1                   0.0   

Year/Month                             0
Box Size                               0
Longitude                              0
Latitude                               0
PID2: Standard/Enhanced                0
Group                                  0
Checksum                               0
S-S1                                   0
S-Median                               0
S-S5                                   0
S-Mean                                 0
S-Observation number                   0
S-Standard deviation                   0
S-Mean day of the month                0
S-Fraction of daylight observations    0
S-Mean Longitude                       0
S-Mean Latitude                        0
A-S1                                   0
A-Median                               0
A-S5                                   0
A-Mean                                 0
A-Observation number                   0
A-Standard deviation                   0
A-Mean day of the month                0
A-Fraction of da

In [ ]:
# Split features and target
X = df[['Year', 'Month', 'Longitude', 'Latitude', 'A-Mean', 'Q-Mean', 'R-Mean']].values  # Features
y = df[['S-Mean']].values  # Targets

# Check for NaN or infinite values in targets
print("NaN values in y:", np.isnan(y).sum())
print("Infinite values in y:", np.isinf(y).sum())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features using RobustScaler
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check for NaN values in scaled features
print("NaN values in X_train:", np.isnan(X_train).sum())
print("NaN values in X_test:", np.isnan(X_test).sum())

print(X_train.shape[1])

In [ ]:
# Define the model
model = Sequential()
model.add(Dense(2, input_dim=X_train.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dense(2, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1]))  # Output layer with 4 neurons for Mean_S, Mean_A, Mean_Q, Mean_R
model.summary()

# Compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.2)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test MAE: {mae}")

# Plot training & validation MAE values
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('Model MAE')
plt.ylabel('Mean Absolute Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model.save('G3_S_model.keras')